# ライブラリのインポート

In [68]:
import numpy as np
import utils as ut
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision.models as models
import multiprocessing

# Discriminator

In [89]:
class Discriminator(nn.Module):
    
    def __init__(self, item_num, user_num, emb_dim,
                           lamda, param=None, initdelta=0.05):
        super(Discriminator, self).__init__()
        self.item_num = item_num
        self.user_num = user_num
        self.emb_dim = emb_dim
        self.lamda = lamda
        self.param = param
        self.initdelta = initdelta
        self.d_params = []
        
        if self.param is None:
            self.user_embeddings = nn.Embedding(
                self.user_num, self.emb_dim)            
            self.item_embeddings = nn.Embedding(
                self.item_num, self.emb_dim)            
            self.item_bias = nn.Embedding(
                self.item_num, 1)
            
            # initialize
            torch.nn.init.uniform_(
                self.user_embeddings.weight, a=-initdelta, b=initdelta)
            torch.nn.init.uniform_(
                self.item_embeddings.weight, a=-initdelta, b=initdelta)
#             torch.nn.init.uniform_(
#                 self.item_bias.weight, a=initdelta, b=-initdelta)
            
    def forward(self, user, item, label):
        pre_logits = (self.user_embeddings(user) \
                              * self.item_embeddings(item)).squeeze().sum(1).view(-1,1) \
                                + self.item_bias(item).view(-1, 1)
        pre_loss =  F.binary_cross_entropy_with_logits(pre_logits, label)
        return pre_loss

    def all_rating(self, user):
        rating = torch.mm(self.user_embeddings(user).view(-1, 5),
                                               self.item_embeddings.weight.t()) + self.item_bias
        return all_rating

    def all_logits(self, user):
        logits = (self.user_embeddings(user) \
                              * self.item_embeddings.weight).sum(1) + self.item_bias
        return all_logits

    def get_reward(self, user, item):
        reward_logits = (self.user_embeddings(user) * self.item_embeddings(item)).squeeze().sum(1).view(-1,1) + self.item_bias(item).view(-1, 1)
        reward = 2 * (torch.sigmoid(reward_logits) - 0.5)
        return reward

# Generator

In [117]:
class Generator(nn.Module):
    
    def __init__(self, item_num, user_num, emb_dim,
                           lamda, param=None, initdelta=0.05, lr=0.05):
        super(Generator, self).__init__()
        self.item_num = item_num
        self.user_num = user_num
        self.emb_dim = emb_dim
        self.lamda = lamda
        self.param = param
        self.initdelta = initdelta
        self.lr = lr
        self.g_params = []
        
        import pickle
        with open("ml-100k/model_dns_ori.pkl", "rb") as f:
            param = pickle.load(f, encoding='latin1')
    
        if self.param is None:
            self.user_embeddings = nn.Embedding(
                self.user_num, self.emb_dim)
            self.item_embeddings = nn.Embedding(
                self.item_num, self.emb_dim)
            self.item_bias = torch.zeros(self.item_num)
            
        self.user_embeddings.weight = torch.nn.Parameter(torch.tensor(param[0]))
        self.item_embeddings.weight = torch.nn.Parameter(torch.tensor(param[1]))
        
            # initialize
#             torch.nn.init.uniform_(
#                 self.user_embeddings.weight, a=-initdelta, b=initdelta)
#             torch.nn.init.uniform_(
#                 self.item_embeddings.weight, a=-initdelta, b=initdelta)
        
    def forward(self, user, item, reward):
        softmax_score = F.softmax(self.all_logits(user).view(1, -1), -1)
        i_prob = torch.gather(softmax_score.view(-1), 0, item).clamp(min=1e-8)
        loss = - torch.mean(torch.log(i_prob) * reward) # \
#                    + self.lamda * (F.normalize(self.user_embeddings(user), p=2, dim=1) \
#                                              + F.normalize(self.item_embeddings(item), p=2, dim=1) \
#                                              + F.normalize(self.item_bias(item), p=2, dim=1))
        return loss

    def all_rating(self, user):
        rating = torch.mm(self.user_embeddings(user).view(-1, 5),
                                               self.item_embeddings.weight.t()) + self.item_bias
        return rating
    
    def all_logits(self, user):
        logits = (self.user_embeddings(user) \
                              * self.item_embeddings.weight).sum(1) + self.item_bias
        return logits

# 評価関数

In [118]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    return np.sum(r / np.log2(np.arange(2, r.size + 2)))


def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

def simple_test_one_user(x):
    # import pdb; pdb.set_trace()
    rating = x[0]
    u = x[1]

    test_items = list(all_items - set(user_pos_train[u]))
    item_score = []
    for i in test_items:
        item_score.append((i, rating[i]))

    item_score = sorted(item_score, key=lambda x: x[1])
    item_score.reverse()
    item_sort = [x[0] for x in item_score]

    r = []
    for i in item_sort:
        if i in user_pos_test[u]:
            r.append(1)
        else:
            r.append(0)

    p_3 = np.mean(r[:3])
    p_5 = np.mean(r[:5])
    p_10 = np.mean(r[:10])
    ndcg_3 = ndcg_at_k(r, 3)
    ndcg_5 = ndcg_at_k(r, 5)
    ndcg_10 = ndcg_at_k(r, 10)

    return np.array([p_3, p_5, p_10, ndcg_3, ndcg_5, ndcg_10])

def simple_test(model):
    result = np.array([0.] * 6)
    pool = multiprocessing.Pool(cores)
    batch_size = 128
    test_users = list(user_pos_test.keys())
    test_user_num = len(test_users)
    index = 0
    while True:
        if index >= test_user_num:
            break
        user_batch = test_users[index:index + batch_size]
        index += batch_size
    
#         print(user_batch)
        user_batch_rating = model.all_rating(torch.tensor(user_batch))

        user_batch_rating = user_batch_rating.detach_().cpu().numpy()

        user_batch_rating_uid = zip(user_batch_rating, user_batch)
        batch_result = pool.map(simple_test_one_user, user_batch_rating_uid)
        for re in batch_result:
            result += re

    pool.close()
    ret = result / test_user_num
    ret = list(ret)
    return ret

# ハイパーパラメーター

In [119]:
EMB_DIM = 5
USER_NUM = 943
ITEM_NUM = 1683
BATCH_SIZE = 16
INIT_DELTA = 0.05

all_items = set(range(ITEM_NUM))
workdir = "ml-100k/"
DIS_TRAIN_FILE = workdir + "dis-train.txt"
cores = multiprocessing.cpu_count()

# ロードデータ

In [120]:
# positiveな要素だけを引っ張ってくる
user_pos_train = {}
with open(workdir + 'movielens-100k-train.txt')as fin:
    for line in fin:
        line = line.split()
        uid = int(line[0])
        iid = int(line[1])
        r = float(line[2])
        if r > 3.99:
            if uid in user_pos_train:
                user_pos_train[uid].append(iid)
            else:
                user_pos_train[uid] = [iid]

# testでnegativeな要素だけを引っ張ってくる                
user_pos_test = {}
with open(workdir + 'movielens-100k-test.txt')as fin:
    for line in fin:
        line = line.split()
        uid = int(line[0])
        iid = int(line[1])
        r = float(line[2])
        if r > 3.99:
            if uid in user_pos_test:
                user_pos_test[uid].append(iid)
            else:
                user_pos_test[uid] = [iid]

all_users = user_pos_train.keys()

# Generatorによるデータセレクト

In [121]:
def generate_for_d(model, filename):
    data = []
    for user in user_pos_train:
        pos = user_pos_train[user]

        rating = model.all_rating(torch.tensor(user))
        rating = rating.detach_().cpu().numpy()
        rating = np.array(rating) / 0.2  # Temperature
        exp_rating = np.exp(rating)
        prob = exp_rating / np.sum(exp_rating)

        neg = np.random.choice(np.arange(ITEM_NUM), size=len(pos), p=prob.reshape(-1,))
        for i in range(len(pos)):
            data.append(str(user) + '\t' + str(pos[i]) + '\t' + str(neg[i]))

    with open(filename, 'w')as fout:
        fout.write('\n'.join(data))

In [122]:
generator = Generator(
    ITEM_NUM, USER_NUM,EMB_DIM, lamda=0.0 / BATCH_SIZE,
    param=None, initdelta=INIT_DELTA)

discriminator = Discriminator(
    ITEM_NUM, USER_NUM,EMB_DIM, lamda=0.0 / BATCH_SIZE,
    param=None, initdelta=INIT_DELTA)

g_optimizer = torch.optim.SGD(
    generator.parameters(), lr=0.001, momentum=0.9)

d_optimizer = torch.optim.SGD(
    discriminator.parameters(), lr=0.001, momentum=0.9)                    

In [ ]:
from torch import nn
from torch import autograd

generator.train()
discriminator.train()
for epoch in range(15):
    if epoch >= 0:
        for d_epoch in range(100):
            if d_epoch % 5 == 0:
                generate_for_d(generator, DIS_TRAIN_FILE)
                train_size = ut.file_len(DIS_TRAIN_FILE)
            index = 1
            while True:
                if index > train_size:
                    break
                if index + BATCH_SIZE <= train_size + 1:
                    users, items, labels = ut.get_batch_data(
                        DIS_TRAIN_FILE, index, BATCH_SIZE)
                else:
                    users, items, labels = ut.get_batch_data(
                        DIS_TRAIN_FILE, index, train_size - index + 1)
                    
                index += BATCH_SIZE
                users = torch.tensor(users).view(-1, 1)
                items = torch.tensor(items).view(-1, 1)
                labels = torch.tensor(labels).view(-1, 1)
    
                loss_d = discriminator(users, items, labels)
                d_optimizer.zero_grad()
                loss_d.backward()
                d_optimizer.step()
            print("\r[D Epoch %d/%d] [loss: %f]" %(d_epoch, 100, loss_d.item()))

        for g_epoch in range(50):
            for user in user_pos_train:
                sample_lambda = 0.2
                pos = user_pos_train[user]

                rating = generator.all_logits(torch.tensor(user))
                rating = rating.detach_().cpu().numpy()
                exp_rating = np.exp(rating)
                prob = exp_rating / np.sum(exp_rating)
                
                pn = (1 - sample_lambda) * prob
                pn[pos] += sample_lambda * 1.0 / len(pos)
                
                sample = np.random.choice(range(ITEM_NUM), 2*len(pos), p=pn)
                reward = discriminator.get_reward(torch.tensor([user] * 2 * len(pos)), torch.tensor(sample))
                reward = reward.detach_().cpu().numpy()
                reward = reward * prob[sample] / pn[sample]

                loss_g = generator(torch.tensor(user), torch.tensor(sample), torch.tensor(reward))
                g_optimizer.zero_grad()
                loss_g.backward()
                g_optimizer.step()
                
            print("\r[G Epoch %d/%d] [loss: %f]" %(g_epoch, 50, loss_g.item()))
            result = simple_test(generator)
            print("epoch ", epoch, "gen: ", result)
#             buf = '\t'.join([str(x) for x in result])
#             gen_log.write(str(epoch) + '\t' + buf + '\n')
#             gen_log.flush()

[D Epoch 0/100] [loss: 0.646515]
[D Epoch 1/100] [loss: 0.646522]
[D Epoch 2/100] [loss: 0.646529]
[D Epoch 3/100] [loss: 0.646536]
[D Epoch 4/100] [loss: 0.646543]
[D Epoch 5/100] [loss: 0.646551]
[D Epoch 6/100] [loss: 0.646559]
[D Epoch 7/100] [loss: 0.646566]
[D Epoch 8/100] [loss: 0.646574]
[D Epoch 9/100] [loss: 0.646582]
[D Epoch 10/100] [loss: 0.646591]
[D Epoch 11/100] [loss: 0.646599]
[D Epoch 12/100] [loss: 0.646607]
[D Epoch 13/100] [loss: 0.646616]
[D Epoch 14/100] [loss: 0.646625]
[D Epoch 15/100] [loss: 0.646633]
[D Epoch 16/100] [loss: 0.646642]
[D Epoch 17/100] [loss: 0.646651]
[D Epoch 18/100] [loss: 0.646660]
[D Epoch 19/100] [loss: 0.646669]
[D Epoch 20/100] [loss: 0.646679]
[D Epoch 21/100] [loss: 0.646688]
[D Epoch 22/100] [loss: 0.646697]
[D Epoch 23/100] [loss: 0.646706]
[D Epoch 24/100] [loss: 0.646715]
[D Epoch 25/100] [loss: 0.646725]
[D Epoch 26/100] [loss: 0.646734]
[D Epoch 27/100] [loss: 0.646744]
[D Epoch 28/100] [loss: 0.646754]
[D Epoch 29/100] [loss: 

[G Epoch 28/50] [loss: 0.235173]
epoch  0 gen:  [0.33187134502924015, 0.3065789473684209, 0.27105263157894705, 0.34321419685315352, 0.32657025711280918, 0.31260383548307924]
[G Epoch 29/50] [loss: 0.225934]
epoch  0 gen:  [0.33187134502924015, 0.30657894736842095, 0.27083333333333304, 0.34321419685315352, 0.32653766221734787, 0.31246287363503827]
[G Epoch 30/50] [loss: 0.216859]
epoch  0 gen:  [0.33187134502924015, 0.30657894736842095, 0.27083333333333304, 0.34321419685315352, 0.32653766221734787, 0.31246287363503827]
[G Epoch 31/50] [loss: 0.089096]
epoch  0 gen:  [0.33187134502924015, 0.30657894736842095, 0.27061403508771897, 0.34359401443003268, 0.32681216647291611, 0.31250148884167334]
[G Epoch 32/50] [loss: 0.397192]
epoch  0 gen:  [0.33187134502924015, 0.30657894736842095, 0.27039473684210497, 0.34359401443003268, 0.32681216647291611, 0.31236196956930723]
[G Epoch 33/50] [loss: -0.160964]
epoch  0 gen:  [0.33187134502924015, 0.30657894736842095, 0.27061403508771897, 0.34359401443

In [116]:
generator.user_embeddings.weight = torch.nn.Parameter(torch.tensor(param[0]))
generator.item_embeddings.weight = torch.nn.Parameter(torch.tensor(param[1]))

In [108]:
generator

Generator(
  (user_embeddings): Embedding(943, 5)
  (item_embeddings): Embedding(1683, 5)
  (item_bias): Embedding(1683, 1)
)